In [25]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
import math
import time
import sqlite3

## Add additional population and closest cities data to Tanzania Water Pump data file

In [26]:
# define column names for text file mapping
colNames = ['geonameid','name','asciiname','alternatenames',
           'latitude','longitude','feature class', 'feature code',
           'country code', 'cc2', 'admin1 code', 'admin2 code',
           'admin3 code', 'admin4 code', 'population', 'elevation',
           'dem', 'timezone', 'modification_date']

In [27]:
# load supplemental data into pandas
pop_df = pd.read_table('./Resources/population_data/TZ.txt', header=None)
pop_df.columns = colNames
pop_df.sort_values(by='population', ascending=False)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
930,149590,United Republic of Tanzania,United Republic of Tanzania,"An Tansain,An Tansáin,Orileede Tansania,Orílẹ́...",-6.00000,35.00000,A,PCLI,TZ,NaN,0.0,NaN,NaN,NaN,56318348,NaN,829,Africa/Dar_es_Salaam,2020-03-29
11530,160260,Dar es Salaam Region,Dar es Salaam Region,"Dar es Salaam,Dar es Salaam Region,Dar-es-Salaam",-6.83523,39.19597,A,ADM1,TZ,NaN,23.0,NaN,NaN,NaN,2791063,NaN,62,Africa/Dar_es_Salaam,2011-03-17
3545,152219,Mwanza Region,Mwanza Region,"Mkoa wa Mwanza,Mwanza,Mwanza Region",-2.75000,33.08333,A,ADM1,TZ,NaN,12.0,NaN,NaN,NaN,2772509,NaN,1206,Africa/Dar_es_Salaam,2015-05-04
5683,154375,Mbeya Region,Mbeya Region,"Mbeya,Mbeya Region,Mkoa wa Mbeya",-8.20000,33.33333,A,ADM1,TZ,NaN,9.0,NaN,NaN,NaN,2707410,NaN,1315,Africa/Dar_es_Salaam,2015-05-04
11533,160263,Dar es Salaam,Dar es Salaam,"DAR,Dar Es Salaam,Dar Es Salam,Dar es Salaam,D...",-6.82349,39.26951,P,PPLA,TZ,NaN,23.0,702.0,702132.0,NaN,2698652,NaN,24,Africa/Dar_es_Salaam,2019-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6524,155223,Majombe,Majombe,NaN,-8.83333,33.96667,P,PPL,TZ,NaN,9.0,1207.0,1207171.0,NaN,0,NaN,1214,Africa/Dar_es_Salaam,2016-07-26
6523,155222,Majonjo,Majonjo,NaN,-5.08333,39.06667,P,PPL,TZ,NaN,18.0,404.0,404032.0,NaN,0,NaN,49,Africa/Dar_es_Salaam,2016-07-26
6522,155221,Majonjo,Majonjo,NaN,-5.20000,39.03333,P,PPL,TZ,NaN,18.0,404.0,404151.0,NaN,0,NaN,59,Africa/Dar_es_Salaam,2016-07-26
6521,155220,Makaange,Makaange,NaN,-5.21667,39.75000,P,PPL,TZ,NaN,20.0,5501.0,5501181.0,NaN,0,NaN,41,Africa/Dar_es_Salaam,2016-07-26


In [28]:
# subset data to include just city level data (admin3 code) and greater than 0 population for ranking
city_pop_df = pop_df[pop_df['admin3 code'].notnull()]
city_pop_df = city_pop_df[city_pop_df['population']>0]

# Create a combined column with lat/lon
city_pop_df['geo_loc'] = city_pop_df[['latitude','longitude']].values.tolist()

city_pop_df.sort_values(by='population', ascending=False)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date,geo_loc
11533,160263,Dar es Salaam,Dar es Salaam,"DAR,Dar Es Salaam,Dar Es Salam,Dar es Salaam,D...",-6.82349,39.26951,P,PPLA,TZ,NaN,23.0,702.0,702132.0,NaN,2698652,NaN,24,Africa/Dar_es_Salaam,2019-09-05,"[-6.82349, 39.26951]"
536,149193,Unguja Ukuu Kaebona,Unguja Ukuu Kaebona,"Unguja Ukuu,Unguja Ukuu Kaebona",-6.27452,39.37486,A,ADM3,TZ,NaN,21.0,5201.0,5201381.0,NaN,1060416,NaN,32,Africa/Dar_es_Salaam,2016-07-25,"[-6.27452, 39.37486]"
73,148726,Zanzibar Island,Zanzibar Island,"Ile de Zanzibar,Menuthias,Unguja,Unguja Island...",-6.15557,39.34170,T,ISL,TZ,NaN,21.0,5201.0,5201181.0,NaN,896721,NaN,39,Africa/Dar_es_Salaam,2019-04-13,"[-6.15557, 39.3417]"
3549,152224,Mwanza,Mwanza,"MWZ,Muansa,Muanza,Muvanzo,Mvanza,Mwamza,Mwansa...",-2.51667,32.90000,P,PPLA,TZ,NaN,12.0,1903.0,1903042.0,NaN,436801,NaN,1144,Africa/Dar_es_Salaam,2019-09-05,"[-2.51667, 32.9]"
77,148730,Zanzibar,Zanzibar,"Ilu Zanzibar,Poli tis Zanzivaris,Sansibar,Sans...",-6.16394,39.19793,P,PPLA,TZ,NaN,25.0,5302.0,5302272.0,NaN,403658,NaN,14,Africa/Dar_es_Salaam,2019-09-05,"[-6.16394, 39.19793]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,156588,Koani Ndogo,Koani Ndogo,Koani Ndogo,-6.13333,39.28333,P,PPL,TZ,NaN,21.0,5201.0,5201093.0,NaN,2311,NaN,44,Africa/Dar_es_Salaam,2018-12-04,"[-6.13333, 39.28333]"
7881,156589,Koani,Koani,Koani,-6.13333,39.28333,P,PPLA,TZ,NaN,21.0,5201.0,5201093.0,NaN,2211,NaN,44,Africa/Dar_es_Salaam,2016-07-26,"[-6.13333, 39.28333]"
3050,151720,Nganane,Nganane,NaN,-6.40000,39.55000,P,PPL,TZ,NaN,21.0,5202.0,5202041.0,NaN,1929,NaN,31,Africa/Dar_es_Salaam,2016-07-26,"[-6.4, 39.55]"
14350,6615413,Hadzabe encampment,Hadzabe encampment,NaN,-3.52810,35.43537,S,HUTS,TZ,NaN,26.0,204.0,204061.0,NaN,20,NaN,1135,Africa/Dar_es_Salaam,2016-07-26,"[-3.5281, 35.43537]"


In [29]:
# Define function to calculate distance between two GPS points - return distance in km

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))/1000

In [30]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("./Resources/Tanzania_Water_Pump.db")
water_df = pd.read_sql_query("SELECT * from pump_it_up_training_set", con)

con.close()

# Create a combined column with lat/lon and drop long where 0
water_df['longitude'] = water_df['longitude'].astype(float)
water_df['geo_loc'] = water_df[['latitude','longitude']].values.tolist()

# View df
water_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,geo_loc
0,0,0.0,2012-11-13,Tasaf,0,TASAF,33.125828,-5.118154,Mratibu,0,...,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,"[-5.11815407, 33.1258283]"
1,1,0.0,2011-03-05,Shipo,1978,SHIPO,34.770717,-9.395642,none,0,...,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,"[-9.39564152, 34.77071669]"
2,2,0.0,2011-03-27,Lvia,0,LVIA,36.115056,-6.279268,Bombani,0,...,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional,"[-6.27926803, 36.11505595]"
3,3,10.0,2013-06-03,Germany Republi,1639,CES,37.147432,-3.187555,Area 7 Namba 5,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,"[-3.18755455, 37.14743219]"
4,4,0.0,2011-03-22,Cmsr,0,CMSR,36.164893,-6.099289,Ezeleda,0,...,good,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,"[-6.09928949, 36.16489341]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,74240,0.0,2013-03-22,World Vision,1183,World vision,37.007726,-3.280868,Upendo Primary School,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,"[-3.28086824, 37.00772596]"
59396,74242,0.0,2011-04-12,Danida,0,DANIDA,33.724987,-8.940758,Kwa Mvulula,0,...,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional,"[-8.94075788, 33.72498733]"
59397,74243,0.0,2012-11-13,Ministry Of Water,1188,Hesawa,33.963539,-1.429477,Kwa Wambura Msege,0,...,good,enough,enough,lake,river/lake,surface,communal standpipe multiple,communal standpipe,non functional,"[-1.42947691, 33.96353891]"
59398,74246,50.0,2011-03-07,Ruthe,1428,Ruthe,35.630481,-7.710549,none,0,...,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe,non functional,"[-7.71054858, 35.63048144]"


In [31]:
# Custom apply function for finding 3 closest city distance and population
def compare_distance_all(ref, city_df):
    
    # use GPS coordinates of current water table and all Cities to calc distance
    city_df['distance'] = city_df['geo_loc'].apply(haversine,args=(ref,))
    
    # Sort a subset dataframe by distance ascending
    sorted_cities = city_df[['distance','population']].sort_values(by=['distance']).iloc[:3]
    
    # Slice custom values for the desired position
    city1 = sorted_cities.iloc[[0]]
    city1.columns = ['distance1','population1']
    
    city2 = sorted_cities.iloc[[1]]
    city2.columns = ['distance2','population2']
    
    city3 = sorted_cities.iloc[[2]]
    city3.columns = ['distance3','population3']
    
    city_series = [city1['distance1'].values[0],city1['population1'].values[0],
                  city2['distance2'].values[0],city2['population2'].values[0],
                  city3['distance3'].values[0],city3['population3'].values[0]]

    # Return as series so it can add a new column
    return pd.Series(city_series)

In [32]:
# Add the three closest cities to the dataframe
water_df[['distance1','population1',
            'distance2','population2',
            'distance3','population3']] = water_df['geo_loc'].apply(compare_distance_all,
                                                            args=(city_pop_df,))

water_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,waterpoint_type,waterpoint_type_group,status_group,geo_loc,distance1,population1,distance2,population2,distance3,population3
0,0,0.0,2012-11-13,Tasaf,0,TASAF,33.125828,-5.118154,Mratibu,0,...,hand pump,hand pump,non functional,"[-5.11815407, 33.1258283]",35.033967,145292.0,43.859263,32900.0,65.659579,15320.0
1,1,0.0,2011-03-05,Shipo,1978,SHIPO,34.770717,-9.395642,none,0,...,hand pump,hand pump,functional,"[-9.39564152, 34.77071669]",5.169916,46724.0,42.274055,19040.0,51.339210,15168.0
2,2,0.0,2011-03-27,Lvia,0,LVIA,36.115056,-6.279268,Bombani,0,...,communal standpipe multiple,communal standpipe,functional,"[-6.27926803, 36.11505595]",27.054749,5527.0,34.494481,11840.0,37.477470,10000.0
3,3,10.0,2013-06-03,Germany Republi,1639,CES,37.147432,-3.187555,Area 7 Namba 5,0,...,communal standpipe,communal standpipe,functional,"[-3.18755455, 37.14743219]",21.108603,22839.0,27.433508,156959.0,38.571507,18726.0
4,4,0.0,2011-03-22,Cmsr,0,CMSR,36.164893,-6.099289,Ezeleda,0,...,hand pump,hand pump,non functional,"[-6.09928949, 36.16489341]",15.549644,5527.0,30.011861,11840.0,42.023175,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,74240,0.0,2013-03-22,World Vision,1183,World vision,37.007726,-3.280868,Upendo Primary School,0,...,communal standpipe,communal standpipe,functional,"[-3.28086824, 37.00772596]",2.722825,22839.0,19.945148,18726.0,23.792744,9568.0
59396,74242,0.0,2011-04-12,Danida,0,DANIDA,33.724987,-8.940758,Kwa Mvulula,0,...,communal standpipe,communal standpipe,functional,"[-8.94075788, 33.72498733]",4.651106,12219.0,24.688760,10318.0,26.972198,10642.0
59397,74243,0.0,2012-11-13,Ministry Of Water,1188,Hesawa,33.963539,-1.429477,Kwa Wambura Msege,0,...,communal standpipe multiple,communal standpipe,non functional,"[-1.42947691, 33.96353891]",19.803600,121119.0,37.506370,16376.0,40.691637,10550.0
59398,74246,50.0,2011-03-07,Ruthe,1428,Ruthe,35.630481,-7.710549,none,0,...,communal standpipe,communal standpipe,non functional,"[-7.71054858, 35.63048144]",9.882770,111820.0,44.921236,29193.0,57.908190,18010.0


In [33]:
# Export to CSV with merged supplemental data
water_df.to_csv("./Resources/water_pump_closest_cities.csv")

In [34]:
# Import the supplemneted CSV into the SQLlite database

# load data
df = pd.read_csv('./Resources/water_pump_closest_cities.csv')

# strip whitespace from headers
df.columns = df.columns.str.strip()

con = sqlite3.connect("./Resources/Tanzania_Water_Pump.db")

# drop data into database
df.to_sql("Water_Pump_Status_Complete", con, if_exists='replace')

con.close()

C:\Users\kenww\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Prep Tanzania Water Pump Training Dataset

In [35]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("./Resources/Tanzania_Water_Pump.db")

train_values_df = pd.read_sql_query("SELECT * from Water_Pump_Status_Complete", con)
train_values_df = train_values_df.set_index("id").drop(['index'],axis=1)

train_labels_df = pd.DataFrame(train_values_df['status_group'])
train_values_df = train_values_df.drop(['status_group','Unnamed: 0'],axis=1)

con.close()

# Verify that result of SQL query is stored in the dataframe
train_values_df

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,source_class,waterpoint_type,waterpoint_type_group,geo_loc,distance1,population1,distance2,population2,distance3,population3
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,2012-11-13,Tasaf,0,TASAF,33.125828,-5.118154,Mratibu,0,Lake Tanganyika,...,groundwater,hand pump,hand pump,"[-5.11815407, 33.1258283]",35.033967,145292.0,43.859263,32900.0,65.659579,15320.0
1,0.0,2011-03-05,Shipo,1978,SHIPO,34.770717,-9.395642,none,0,Rufiji,...,groundwater,hand pump,hand pump,"[-9.39564152, 34.77071669]",5.169916,46724.0,42.274055,19040.0,51.339210,15168.0
2,0.0,2011-03-27,Lvia,0,LVIA,36.115056,-6.279268,Bombani,0,Wami / Ruvu,...,groundwater,communal standpipe multiple,communal standpipe,"[-6.27926803, 36.11505595]",27.054749,5527.0,34.494481,11840.0,37.477470,10000.0
3,10.0,2013-06-03,Germany Republi,1639,CES,37.147432,-3.187555,Area 7 Namba 5,0,Pangani,...,groundwater,communal standpipe,communal standpipe,"[-3.18755455, 37.14743219]",21.108603,22839.0,27.433508,156959.0,38.571507,18726.0
4,0.0,2011-03-22,Cmsr,0,CMSR,36.164893,-6.099289,Ezeleda,0,Wami / Ruvu,...,groundwater,hand pump,hand pump,"[-6.09928949, 36.16489341]",15.549644,5527.0,30.011861,11840.0,42.023175,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74240,0.0,2013-03-22,World Vision,1183,World vision,37.007726,-3.280868,Upendo Primary School,0,Pangani,...,surface,communal standpipe,communal standpipe,"[-3.28086824, 37.00772596]",2.722825,22839.0,19.945148,18726.0,23.792744,9568.0
74242,0.0,2011-04-12,Danida,0,DANIDA,33.724987,-8.940758,Kwa Mvulula,0,Rufiji,...,surface,communal standpipe,communal standpipe,"[-8.94075788, 33.72498733]",4.651106,12219.0,24.688760,10318.0,26.972198,10642.0
74243,0.0,2012-11-13,Ministry Of Water,1188,Hesawa,33.963539,-1.429477,Kwa Wambura Msege,0,Lake Victoria,...,surface,communal standpipe multiple,communal standpipe,"[-1.42947691, 33.96353891]",19.803600,121119.0,37.506370,16376.0,40.691637,10550.0


In [36]:
#create an integer lookup for the status group
dict = {
    "functional":0,
    "functional needs repair":1,
    "non functional":2
}

In [37]:
# convert dates from object to datetime
train_values_df['date_recorded'] = pd.to_datetime(train_values_df['date_recorded'])

In [38]:
# for testing, fillNA with "OTHER" or zero
for col in train_values_df.columns:
    
    if train_values_df[col].dtype == "object":
            train_values_df[col] = train_values_df[col].fillna("Other")
            
    elif train_values_df[col].dtype == "int64":
        train_values_df[col] = train_values_df[col].fillna("0").astype('int64')
        
    elif train_values_df[col].dtype == "float64":
        train_values_df[col] = train_values_df[col].fillna("0").astype('float64')

In [39]:
# Generate categorical variable list
txt_cols = train_values_df.dtypes[train_values_df.dtypes == "object"].index.tolist()

In [40]:
# Check the number of unique values in each column
train_values_df[txt_cols].nunique()

funder                    1898
installer                 2146
wpt_name                 37400
basin                        9
subvillage               19288
region                      21
lga                        125
ward                      2092
recorded_by                  1
scheme_management           12
scheme_name               2697
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
geo_loc                  57520
dtype: int64

In [41]:
many_uniques = ['funder','installer',
                'subvillage','lga','ward', 'scheme_name']

# Determine which values to replace
bucket_df = train_values_df.copy()

for i in many_uniques:
    list_value_counts = bucket_df[i].value_counts()
    
    filter_value = list_value_counts.quantile(0.95)
    
    replace_list = list(list_value_counts[list_value_counts < 100].index)
    
    for j in replace_list:
        bucket_df[i] = bucket_df[i].replace(j,"Other")
    
    print(f"completed {i}")

completed funder
completed installer
completed subvillage
completed lga
completed ward
completed scheme_name


In [42]:
# Check the number of unique values in each column after bucketing
bucket_df[txt_cols].nunique()

funder                      92
installer                   85
wpt_name                 37400
basin                        9
subvillage                  23
region                      21
lga                        116
ward                        59
recorded_by                  1
scheme_management           12
scheme_name                 33
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
geo_loc                  57520
dtype: int64

In [43]:
# drop name and duplicate columns
cleaned_df = bucket_df.drop(['wpt_name','payment_type','quality_group',
                            'quantity_group','source_type',
                            'waterpoint_type_group','date_recorded','geo_loc'], axis=1)

In [45]:
# Finish ETL process with final load into the SQLite db

con = sqlite3.connect("./Resources/Tanzania_Water_Pump.db")


# drop data into database
cleaned_df.to_sql("Water_Pump_Features_ETL", con, if_exists="replace")
train_labels_df.to_sql("Water_Pump_Labels_ETL", con, if_exists="replace")

con.close()